In [2]:
import os

from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity

from langchain_openai import ChatOpenAI, AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate

from langchain_community.document_loaders import SRTLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain_chroma import Chroma

from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


In [3]:
load_dotenv()

### Empower ChatGPT with File Search capabilities.
Create a vector representation of text, store it, and use it for information retrieval.

#### Text Embeddings
Embedding take a piece of text and create a numerical representation of it, which is also known as a "word-to-vector".
 
![embeddings](https://python.langchain.com/v0.2/assets/images/embeddings-9c2616450a3b4f497a2d95a696b5f1a7.png)  
Reference: https://python.langchain.com/v0.2/docs/concepts/#embedding-models

In [4]:
embeddings_model = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    deployment="text-embedding-ada-002-1"
)

In [5]:
# Embed a single piece of text.
query = "天空是藍色的"
embedded_query = embeddings_model.embed_query(query)

print(f'Original query: {query}')
print(f'Embedded query: {embedded_query}')
print(f'Feature length: 共計 {len(embedded_query)} 個特徵/維度')

Original query: 天空是藍色的
Embedded query: [0.01486588828265667, -0.01656484790146351, 0.000708037696313113, -0.026944423094391823, -0.032598771154880524, 0.014666792005300522, -0.021767908707261086, 0.0011696931906044483, 0.0003484192711766809, -0.020759152248501778, -0.009974746033549309, 0.019604390487074852, -0.008979261852800846, -0.008574431762099266, -0.014149140566587448, 0.00045211546239443123, 0.011720160022377968, 0.00028060193290002644, 0.00042391009628772736, -0.004632316995412111, -0.00037475809222087264, 0.036607250571250916, 0.01486588828265667, 0.013233295641839504, 0.0018333490006625652, 0.004406674299389124, 0.022458110004663467, -0.03188202157616615, 0.024435805156826973, -0.000750345760025084, 0.005518297664821148, 0.0007424648501910269, -0.01890091598033905, -0.013989862985908985, 0.00994819961488247, -0.008959352970123291, 0.0004305466718506068, 0.007685133721679449, -0.0072006648406386375, -0.0025152552407234907, 0.014268598519265652, 0.008647434413433075, 0.0064208

In [6]:
# Embed a list of texts.
queries = [
    "天空是藍色的",
    "天空不是紅色的",
    "莓果是藍色的",
    "sky is blue",
    "Betty 是一隻貓",
]
embedded_queries = embeddings_model.embed_documents(queries)

print(f'Original queries: {queries}')
print(f'Embedded queries: {embedded_queries}')
print(f'Feature length: 共計 {len(embedded_queries)} * {len(embedded_queries[0])} 個特徵/維度')

Original queries: ['天空是藍色的', '天空不是紅色的', '莓果是藍色的', 'sky is blue', 'Betty 是一隻貓']
Embedded queries: [[0.01486588828265667, -0.01656484790146351, 0.000708037696313113, -0.026944423094391823, -0.032598771154880524, 0.014666792005300522, -0.021767908707261086, 0.0011696931906044483, 0.0003484192711766809, -0.020759152248501778, -0.009974746033549309, 0.019604390487074852, -0.008979261852800846, -0.008574431762099266, -0.014149140566587448, 0.00045211546239443123, 0.011720160022377968, 0.00028060193290002644, 0.00042391009628772736, -0.004632316995412111, -0.00037475809222087264, 0.036607250571250916, 0.01486588828265667, 0.013233295641839504, 0.0018333490006625652, 0.004406674299389124, 0.022458110004663467, -0.03188202157616615, 0.024435805156826973, -0.000750345760025084, 0.005518297664821148, 0.0007424648501910269, -0.01890091598033905, -0.013989862985908985, 0.00994819961488247, -0.008959352970123291, 0.0004305466718506068, 0.007685133721679449, -0.0072006648406386375, -0.002515255240723

In [7]:
# Semantic Similarity.
for a_query, a_embedded_query in zip(queries, embedded_queries):
    similarity = cosine_similarity([embedded_query], [a_embedded_query])[0][0]
    print(f'The cosine similarity between "{query}" and "{a_query}" is {round(similarity, 4)}.')

The cosine similarity between "天空是藍色的" and "天空是藍色的" is 1.0.
The cosine similarity between "天空是藍色的" and "天空不是紅色的" is 0.9218.
The cosine similarity between "天空是藍色的" and "莓果是藍色的" is 0.8861.
The cosine similarity between "天空是藍色的" and "sky is blue" is 0.851.
The cosine similarity between "天空是藍色的" and "Betty 是一隻貓" is 0.7815.


#### Vector Stores
Vector stores are databases that can efficiently store and retrieve embeddings.

Reference: https://python.langchain.com/v0.2/docs/concepts/#vector-stores

In [8]:
# Step 1: Load the document.
loader = SRTLoader(f'../data/sth_subtitles.srt')
raw_documents = loader.load()
raw_documents

[Document(metadata={'source': '../data/sth_subtitles.srt'}, page_content='另外呢,還有一個最重要的參數叫做temperature 這個temperature呢,它就是一個控制那個選詞語的溫度 這樣講你可能覺得很莫名其妙 你可以這樣想,就是這個確認GPT它在運作的時候 其實就是一個文字接龍的一個引擎 你給它一段話,它就會去從可以接在這段話後面的token裡面去挑 哪一個token比較適合接 所以每一個token都會有它的,我用比較白話的方式講叫分數 就是分數越高的,表示那個接在後面越恰當 那這個temperature它就是去控制 我們如果把那個可以接在後面的字的token 當成在一鍋鍋子裡面煮的東西的話 那個temperature就是控制這個鍋子底下的爐火的大小 那理論上如果你都不去動它的話 分數越高的會浮在越上面 分數越低的會沉在底下 所以如果你都不去動它,它就比較可能會跳到那個上面分數比較高的詞語 那這樣子它的回話就會很固定 那這個temperature就是你可以控制那個火候 所以如果你把那個溫度加大 它就會滾開這個鍋子裡面的東西 所以就像你煮東西一樣 你在滾的時候它是不是就會翻來翻去 那這個時候即使是底下那個分數比較低的token 也有可能會翻到上面被選到 那這個temperature就是一個0到2的值 0的話你就把它當成就是把爐火關掉 2的話就是開大火 0到2中間你自己可以去調 所以你可以做個實驗 像這裡我左邊是temperature是0 右邊是temperature是2 不過你可能會遇到那個temperature設成2的時候 如果你會遇到那個它怎麼跑都沒結果的話 你就把它改成1.9 應該就可以了 像這裡我溫度設成0的時候 你就會發現 我叫它回我兩句話 兩句話都幾乎一樣 那我如果把溫度設成2的時候 讓爐火滾一下 你就會發現它回我兩句話就會不大一樣 那這個就是你用temperature去控制 所以在API的文件上它會說 temperature會控制那個回話的豐富度 因為你如果設成0 你問同樣的話它幾乎都會回一樣的內容 那你設成2它就會有點不一樣 那這個是temperature 那另外有一個參數叫做TopN TopN跟temperature有點像 但是TopN的意思是 它只取那個 TopN是

In [9]:
# Step 2: Split the documents into smaller chunks.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
documents = text_splitter.split_documents(raw_documents)
for doc in documents:
    print(doc)

page_content='另外呢,還有一個最重要的參數叫做temperature 這個temperature呢,它就是一個控制那個選詞語的溫度 這樣講你可能覺得很莫名其妙 你可以這樣想,就是這個確認GPT它在運作的時候 其實就是一個文字接龍的一個引擎 你給它一段話,它就會去從可以接在這段話後面的token裡面去挑 哪一個token比較適合接 所以每一個token都會有它的,我用比較白話的方式講叫分數' metadata={'source': '../data/sth_subtitles.srt'}
page_content='就是分數越高的,表示那個接在後面越恰當 那這個temperature它就是去控制 我們如果把那個可以接在後面的字的token 當成在一鍋鍋子裡面煮的東西的話 那個temperature就是控制這個鍋子底下的爐火的大小 那理論上如果你都不去動它的話 分數越高的會浮在越上面 分數越低的會沉在底下 所以如果你都不去動它,它就比較可能會跳到那個上面分數比較高的詞語 那這樣子它的回話就會很固定' metadata={'source': '../data/sth_subtitles.srt'}
page_content='那這個temperature就是你可以控制那個火候 所以如果你把那個溫度加大 它就會滾開這個鍋子裡面的東西 所以就像你煮東西一樣 你在滾的時候它是不是就會翻來翻去 那這個時候即使是底下那個分數比較低的token 也有可能會翻到上面被選到 那這個temperature就是一個0到2的值 0的話你就把它當成就是把爐火關掉 2的話就是開大火 0到2中間你自己可以去調 所以你可以做個實驗' metadata={'source': '../data/sth_subtitles.srt'}
page_content='所以你可以做個實驗 像這裡我左邊是temperature是0 右邊是temperature是2 不過你可能會遇到那個temperature設成2的時候 如果你會遇到那個它怎麼跑都沒結果的話 你就把它改成1.9 應該就可以了 像這裡我溫度設成0的時候 你就會發現 我叫它回我兩句話 兩句話都幾乎一樣 那我如果把溫度設成2的時候 讓爐火滾一下 你就會發現它回我兩句話就會不大一樣' metadata={'source': '../data/s

In [10]:
# Step 3: Embed each chunk and load it into the vector store.
embeddings_model = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    deployment="text-embedding-ada-002-1"
)
db = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model
)

In [11]:
# Step 4: Query the vector store. Search for the most similar chunks to the query.
# Step 4-1: Similarity search.
query = "temperature 的豐富度是多少?"
docs = db.similarity_search(query, k=1)
print(docs[0].page_content)

那這個就是你用temperature去控制 所以在API的文件上它會說 temperature會控制那個回話的豐富度 因為你如果設成0 你問同樣的話它幾乎都會回一樣的內容 那你設成2它就會有點不一樣 那這個是temperature 那另外有一個參數叫做TopN TopN跟temperature有點像 但是TopN的意思是 它只取那個 TopN是一個0到1的數值 它就是一個百分比


In [12]:
# Step 4-2: Similarity search by vector.
embedding_vector =embeddings_model.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

那這個就是你用temperature去控制 所以在API的文件上它會說 temperature會控制那個回話的豐富度 因為你如果設成0 你問同樣的話它幾乎都會回一樣的內容 那你設成2它就會有點不一樣 那這個是temperature 那另外有一個參數叫做TopN TopN跟temperature有點像 但是TopN的意思是 它只取那個 TopN是一個0到1的數值 它就是一個百分比


In [13]:
# Step 4-3: Similarity search with score.
doc_with_scores = db.similarity_search_with_score(query, k=4)  # Get the top 4 most similar chunks.
for doc, score in doc_with_scores:
    print(round(score, 4), doc.page_content)

0.3248 那這個就是你用temperature去控制 所以在API的文件上它會說 temperature會控制那個回話的豐富度 因為你如果設成0 你問同樣的話它幾乎都會回一樣的內容 那你設成2它就會有點不一樣 那這個是temperature 那另外有一個參數叫做TopN TopN跟temperature有點像 但是TopN的意思是 它只取那個 TopN是一個0到1的數值 它就是一個百分比
0.3523 那這個temperature就是你可以控制那個火候 所以如果你把那個溫度加大 它就會滾開這個鍋子裡面的東西 所以就像你煮東西一樣 你在滾的時候它是不是就會翻來翻去 那這個時候即使是底下那個分數比較低的token 也有可能會翻到上面被選到 那這個temperature就是一個0到2的值 0的話你就把它當成就是把爐火關掉 2的話就是開大火 0到2中間你自己可以去調 所以你可以做個實驗
0.3588 所以你可以做個實驗 像這裡我左邊是temperature是0 右邊是temperature是2 不過你可能會遇到那個temperature設成2的時候 如果你會遇到那個它怎麼跑都沒結果的話 你就把它改成1.9 應該就可以了 像這裡我溫度設成0的時候 你就會發現 我叫它回我兩句話 兩句話都幾乎一樣 那我如果把溫度設成2的時候 讓爐火滾一下 你就會發現它回我兩句話就會不大一樣
0.3706 就是分數越高的,表示那個接在後面越恰當 那這個temperature它就是去控制 我們如果把那個可以接在後面的字的token 當成在一鍋鍋子裡面煮的東西的話 那個temperature就是控制這個鍋子底下的爐火的大小 那理論上如果你都不去動它的話 分數越高的會浮在越上面 分數越低的會沉在底下 所以如果你都不去動它,它就比較可能會跳到那個上面分數比較高的詞語 那這樣子它的回話就會很固定


#### RAG, Retrieval-Augmented Generation
Retrievers are responsible for taking a query and returning relevant documents.  
Reference: https://python.langchain.com/v0.2/docs/concepts/#retrievers

![langchain-app-in-its-most-basic-form](https://cdn.ttgtmedia.com/rms/onlineimages/a_langchain_app_in_its_most_basic_form-f.png)  
Reference: https://www.techtarget.com/searchenterpriseai/definition/LangChain

![](https://alphasec.io/content/images/size/w1600/2023/04/image-5.png)
Reference: [Summarize Documents with LangChain and Chroma](https://alphasec.io/summarize-documents-with-langchain-and-chroma/)

In [14]:
# Step 1: Load and split the document.
loader = PyPDFLoader(f'../data/north_taiwan_travel_intro.pdf')
documents = loader.load_and_split()
documents

[Document(metadata={'source': '../data/north_taiwan_travel_intro.pdf', 'page': 0}, page_content='國際亮點 及優良\n      觀光工廠\n參山國家風景區之 獅頭山 風景區   F7\n獅山遊客中心/新竹縣峨眉鄉七星村六寮60-8號  03-5809296\n南庄遊客中心/苗栗縣南庄鄉東村4鄰大同路43號  037-824570\n範圍橫跨新竹縣峨眉、北埔、竹東與苗栗縣南庄、三灣等五鄉鎮，包\n括獅頭山、五指山、南庄遊憩系統等觀光資源；人文景觀以獅頭山、\n五指山寺廟群、客家文化、賽夏．泰雅原住民文化為主；自然景觀以\n獅頭山之巨型岩洞地質、五指山獨特山形及東河石壁雄風最具特色，\n其中獅頭山與五指山晨鐘暮鼓、山靈毓秀，成為北台灣著名的佛教聖\n山；而賽夏族的向天湖矮靈祭是原住民祭典中最特殊者，儀式充滿著\n神秘風采，總是吸引遊客的好奇。\n大眾運輸  \n1.自新竹搭乘臺鐵內灣線至竹東站下車，轉乘新竹客運至沿線各景點站下車，\n  循指標抵各景點。\n2.高鐵新竹站下車，轉乘台灣好行至獅山遊客中心。\n3.自新竹、竹南或頭份搭乘新竹或苗栗客運至北埔、南庄或竹東下車，循指標抵各景點。\n4.自臺鐵竹南站下車，轉乘台灣好行至南庄遊客中心。\n 【新竹客運新竹站 03-5259599．苗栗客運頭份站 037-662111】\n開車  1.國道3號→寶山交流道→竹43→台3→峨眉→北埔→竹東\n      2.國道1號→頭份交流道→縣124甲→縣124→台3→縣124→南庄遊客中心\n      3.國道3號→竹林交流道→縣120→台3→縣122\n        →竹37-4→東河戰備道路→東河  \n雪、堆雪人的樂趣，適合闔家\n親子相伴同遊，是一處充滿知\n性、智慧、教育、遊樂的育樂\n天地。\n大眾運輸  \n1.臺鐵區間車於新豐站下車，轉\n  搭計程車至園區。\n2.高鐵新竹站下車，轉搭計程車\n  至園區。\n開車  \n南下 \n1.國道1號→湖口交流道→台1→明新科技大學右轉→循指標可抵\n2.台15→觀音、永安→老姜的店紅綠燈左轉→竹5(康樂路)→循指\n  標可抵達\n北上  \n1.國道1號→竹北交流道→光明六路過地下道→台1→明新科技大學\n  →上

In [16]:
# Step 2: Embed the documents and load them into the vector store.
embeddings_model = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    deployment="text-embedding-ada-002-1"
)

index = VectorstoreIndexCreator(
    embedding=embeddings_model,
    vectorstore_cls=Chroma,
    vectorstore_kwargs={"persist_directory": "../data/vectorstore"}
).from_loaders([loader])

db = Chroma(
    persist_directory="../data/vectorstore",
    embedding_function=embeddings_model
)

retriever = db.as_retriever(search_kwargs={'k': 3})

In [17]:
# Step 3: Create a retrieval chain, which performed natural-language question answering over a data source using RAG.
llm_model = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-120",
    temperature=0,
)

prompt_template = ChatPromptTemplate.from_messages([
    ("system", (
        "You are an assistant for question-answering tasks."
        "Use the following pieces of retrieved context to answer the question."
        "If you don't know the answer, say that you don't know."
        "Use three sentences maximum and keep the answer concise."
        "請根據上下文來回答問題，不知道答案就回答不知道不要試圖編造答案。"
        "你是一位旅遊助理，請根據景點資訊回覆使用者的問題。"
        "{context}"
    )),
    ("human", "{input}"),
])

combine_docs_chain = create_stuff_documents_chain(llm_model, prompt_template)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [ ]:
query = "嘉義檜意森活村在哪裡?"
response = rag_chain.invoke({"input": query})
response['answer']

In [ ]:
query = "新竹市玻璃工藝博物館有什麼好玩?"
response = rag_chain.invoke({"input": query})
response['answer']

In [ ]:
query = "內灣老街商圈該怎麼去?"
response = rag_chain.invoke({"input": query})
response['answer']

In [20]:
query = "新竹市玻璃工藝博物館有什麼好玩?"
retriever.invoke(query)

[Document(metadata={'page': 0, 'source': '../data/north_taiwan_travel_intro.pdf'}, page_content='新竹市玻璃工藝博物館  B3\n新竹市東大路1段2號  03-5626091\n週二至週日 09：00～17：00．週一、民俗節日及選舉日休館\n新竹市影像博物館  A3       \n新竹市中正路65號  03-5285840     \n13：00～24：00 週二休館，詳細營業時間請電洽詢問或至「或者光盒子」臉書查詢。       \n新竹市眷村博物館  A2           \n新竹市東大路2段105號  03-5338442\n週二至週日 09：00～17：00(週一、民俗節日及選舉日休館)\n清泉張學良故居  F6  \n張學良故居：新竹縣五峰鄉桃山村清泉256-6號  03-5856613(團體預約導覽)\n原住民族館(舊張學良故居):新竹縣五峰鄉桃山村清泉164之6號  03-5851217\n週二至週日 09：00～17：00(週一及園區另行公告休館日)\n黑蝙蝠中隊文物陳列館  A2\n新竹市東大路2段16號 03-5425061\n週二至週日 09：00～17：00\n(週一、民俗節日及選舉日休館)\n(本欄資訊僅供參考，如有調整，請詳閱各館所網站公告為準。)▎拉拉山生態教育館 H6\n地址：桃園市復興區華陵里巴陵207號\n電話：03-3912142\n開放： 06:00～17:00\n▎羅東自然教育中心 K6\n地址：宜蘭縣羅東鎮中正北路118號\n電話：03-9540823\n開放：平日,假日 08:00～17:00\n▎羅東林業文化園區 K6\n地址：宜蘭縣羅東鎮中正北路118號\n電話：03-9549114\n開放：園區 06:00～19:00  \n      展示館 09:00～12:00、\n             14:00～17:00\n      森產館、森動館(週三至週日)\n      ，森活館、竹探館(週五至週二)\n▎員山生態教育館 J6\n開放：展館營運轉型整修中，開放資\n      訊請關注台灣山林悠遊網。\n▎南澳生態教育館 K8\n地址：宜蘭縣南澳鄉中正路55號\n電話：03-9981060\n開放：週一至

In [ ]:
query = "嘉義檜意森活村在哪裡?"
result = index.query_with_sources(query, llm=llm_model)

In [ ]:
query = "新竹市玻璃工藝博物館有什麼好玩?"
result = index.query(query, llm=llm_model)